##Esse arquivo possui como objetivo fazer a quantificação dos dados de resíduos de culturas e da produção de Biochar aplicando conhecimentos de análise e transformação de dados. Os dados presente foram extraídos das fontes especificadas no artigo de estudo e alocados no Drive


## Preparo de Ambiente

In [ ]:
#importar bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Conectar ao Drive para trabalhar com os dados nele presentes
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Análise exploratória, limpeza e filtragens na tabela de resíduos

In [ ]:
#ler a tabela
dfCrop = pd.read_excel('/content/drive/MyDrive/Analise Lefebvre/Dados/CropsProduction.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Analise Lefebvre/Dados/CropsProduction.xlsx'

In [ ]:
#visualização inicial
dfCrop

In [ ]:
dfCrop['Area'].unique().shape

In [ ]:
dfCrop = dfCrop.query('Flag != "M" ')

In [ ]:
cropsLista = [
    "sugar cane",
    "Maize (corn)",
    "Rice",
    "Wheat",
    "Potatoes",
    "Oil palm",
    "Soy beans",
    "Cassava",
    "Sugar beets",
    "Tomatoes",
    "Barley",
    "Sweet potatoes",
    "Apples",
    "Bananas",
    "Grapes",
    "Seed cotton, unginned",
    "Rape or colza seed",
    "Oranges",
    "Groundnuts, excluding shelled",
    "Coconuts, in shell",
    "Tangerines, mandarins, clementines",
    "Sorghum",
    "Millet",
    "Oats",
    "Lemons and limes",
    "Olives",
    "Rye",
    "Coffee, green",
    "Dates",
    "Jute, raw or retted",
    "Green corn (maize)",
    "Unmanufactured tobacco",
    "Beans, dry",
    "Peas, green",
    "Peas, dry",
    "Cow peas, dry",
    "Broad beans and horse beans, dry",
    "Pigeon peas, dry",
    "Broad beans and horse beans, green",
    "String beans",
    "Chick peas, dry",
    "Lentils, dry",
    "Cocoa beans"
]
len(cropsLista)

In [ ]:
# filtrar culturas na base de dados
dfCrop = dfCrop[dfCrop['Item'].isin(cropsLista)]

In [ ]:
dfCrop.head()

In [ ]:
dfCrop.columns

In [ ]:
# Remodelando o DataFrame
df_remoldedCrop = dfCrop.pivot_table(index=['Area', 'Item'], columns='Element', values=['Value', 'Unit'], aggfunc='first').reset_index()

# Renomeando as colunas
df_remoldedCrop.columns = ['_'.join(col).rstrip('_') for col in df_remoldedCrop.columns.values]
df_remoldedCrop = df_remoldedCrop.rename(columns={'Area_': 'Área', 'Production_Value': 'Produção_Value', 'Area_Unit': 'Área_Unit', 'Production_Unit': 'Produção_Unit'})


In [ ]:
df_remoldedCrop

## Análise exploratória, filtragens e transformação dos dados de cada cultura




In [ ]:
dfCropsRatios =  pd.read_excel('/content/drive/MyDrive/Analise Lefebvre/Dados/Tabela 1 dados suplementares 1.xlsx')

In [ ]:
dfCropsRatios.columns

In [ ]:
dfCropsRatios = dfCropsRatios.rename(columns={" Crop": "Crops",
    "Residue": "Residues",
    "Residue type": "Residues Type",
    "Residue-to-product ratio (RPR) [t tproduct-1]": "RPR",
    "Areal residue production  (ARP) [t ha-1]": "ARP",
    "Collection factor [%]": "Collection Factor",
    "Moisture content [%]": "Moisture Content",
    "Biochar organic carbon content  (DM basis) [%]": "Biochar Carbon Content DM basis"
})

In [ ]:
dfCropsRatios

In [ ]:
dfCropsRatios.columns

In [ ]:
dfCropsRatios.query('RPR > 3')

In [ ]:
# Selecionar as linhas onde a coluna 'RPR' é maior que 4, que não condiz com a realidade
linhas_a_dividir = dfCropsRatios['RPR'] > 4

# Substituir os valores na coluna 'RPR' por mil apenas nas linhas onde a condição é verdadeira
dfCropsRatios.loc[linhas_a_dividir, 'RPR'] /= 1000



In [ ]:
#transformar porcentagem em decimal para cálculos
dfCropsRatios[['Collection Factor', 'Moisture Content','Biochar Carbon Content DM basis']] /= 100

In [ ]:
dfCropsRatios

In [ ]:
dfCropsRatios.head()

In [ ]:
df_remoldedCrop.columns

In [ ]:
dfCropsRatios.columns

## Realizar cálculos e filtrar os resultados dos resíduos e da produção de biochar

In [ ]:

# Inicializar uma lista para armazenar os resultados
resultados = []

#considerado rendimento de 0.25 conforme proposto no artigo

# Iterar sobre os países
for pais in df_remoldedCrop['Area'].unique():
    # Filtrar as linhas correspondentes ao país atual
    df_producao_pais = df_remoldedCrop[df_remoldedCrop['Area'] == pais]

    # Inicializar a soma dos resíduos para o país atual
    soma_residuos_pais = 0
    soma_biochar = 0

    # Iterar sobre as culturas do país atual
    for cultura in df_producao_pais['Item']:
        # Filtrar as linhas correspondentes à cultura atual no DataFrame de resíduos
        df_residuos_cultura = dfCropsRatios[dfCropsRatios['Crop'] == cultura]

        # Iterar sobre os tipos de resíduos da cultura atual
        for index, row in df_residuos_cultura.iterrows():
            # Somar os resíduos considerando os coeficientes correspondentes do tipo de resíduo

            if not pd.isna(row['RPR']) and not pd.isna(df_producao_pais[df_producao_pais['Item'] == cultura]['Unit_Production'].values[0]) and not pd.isna(df_producao_pais[df_producao_pais['Item'] == cultura]['Value_Production'].values[0]) :

              soma_biochar += 0.25 *  row['RPR'] * row['Collection Factor'] *( 1 - row['Moisture Content'] )  * df_producao_pais[df_producao_pais['Item'] == cultura]['Value_Production'].values[0]

              soma_residuos_pais += row['RPR'] *( 1 - row['Moisture Content'] ) * row['Collection Factor']   * df_producao_pais[df_producao_pais['Item'] == cultura]['Value_Production'].values[0]

            elif not pd.isna(row['ARP']) and not pd.isna(df_producao_pais[df_producao_pais['Item'] == cultura]['Unit_Area harvested'].values[0]) and not pd.isna(df_producao_pais[df_producao_pais['Item'] == cultura]['Value_Area harvested'].values[0]):

              soma_biochar +=  0.25 * row['ARP'] * row['Collection Factor'] * (1 - row['Moisture Content'] )  * df_producao_pais[df_producao_pais['Item'] == cultura]['Value_Area harvested'].values[0]

              soma_residuos_pais += row['ARP'] *( 1 - row['Moisture Content'] ) * row['Collection Factor']  * df_producao_pais[df_producao_pais['Item'] == cultura]['Value_Area harvested'].values[0]

            else:

              soma_biochar += 0
              soma_residuos_pais += 0



    # Armazenar o resultado para o país atual na lista
    resultados.append({'País': pais, 'Soma de Resíduos': soma_residuos_pais, 'Biochar Produzido': soma_biochar})

# Criar um DataFrame a partir da lista de resultados
df_resultados = pd.DataFrame(resultados)



In [ ]:
dfCropsRatios.columns

In [ ]:
df_resultados

In [ ]:
df_resultados['Soma de Resíduos'].isna().sum()

In [ ]:
countries = [
    'Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina', 'Armenia', 'Australia',
    'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Belize',
    'Benin', 'Bhutan', 'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina',
    'Botswana', 'Brazil', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon',
    'Canada', 'Chad', 'Chile', 'China', 'Colombia', 'Democratic Republic of the Congo',
    'Congo', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Czechia', 'Denmark',
    'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea',
    'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Finland', 'France', 'Gabon', 'Gambia',
    'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana',
    'Haiti', 'Honduras', 'Hungary', 'India', 'Indonesia', 'Iran (Islamic Republic of)', 'Iraq',
    'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya',
    "Democratic People's Republic of Korea", 'Republic of Korea', 'Kuwait', 'Kyrgyzstan',
    "Lao People's Democratic Republic", 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya',
    'Lithuania', 'Madagascar', 'Malawi', 'Malaysia', 'Mali', 'Mauritania', 'Mexico',
    'Republic of Moldova', 'Mongolia', 'Montenegro', 'Morocco', 'Mozambique', 'Myanmar',
    'Namibia', 'Nepal', 'Netherlands (Kingdom of the)', 'New Zealand', 'Nicaragua', 'Niger',
    'Nigeria', 'North Macedonia', 'Norway', 'Oman', 'Pakistan', 'Panama', 'Paraguay', 'Peru',
    'Philippines', 'Poland', 'Portugal', 'Qatar', 'Romania', 'Russian Federation', 'Rwanda',
    'Saudi Arabia', 'Senegal', 'Serbia', 'Sierra Leone', 'Slovakia', 'Slovenia', 'Somalia',
    'South Africa', 'South Sudan', 'Spain', 'Sri Lanka', 'Sudan', 'Sweden', 'Switzerland',
    'Syrian Arab Republic', 'Tajikistan', 'United Republic of Tanzania', 'Thailand', 'Timor-Leste',
    'Togo', 'Trinidad and Tobago', 'Tunisia', 'Türkiye', 'Turkmenistan', 'Uganda', 'Ukraine',
    'United Arab Emirates', 'United Kingdom of Great Britain and Northern Ireland',
    'United States of America', 'Uruguay', 'Uzbekistan', 'Venezuela (Bolivarian Republic of)',
    'Viet Nam', 'Yemen', 'Zambia', 'Zimbabwe'
]



In [ ]:
df_filtrado = df_resultados[df_resultados['País'].isin(countries)]

In [ ]:
df_filtrado.sort_values(by='País')
df_filtrado

In [ ]:
#salvar resultado em excel
df_filtrado.to_excel("/content/drive/MyDrive/Analise Lefebvre/Dados quantificados/residues crops.xlsx", index=True)

In [ ]:
len(df_filtrado)